In [ ]:
import malariagen_data

In [ ]:
import dask
dask.config.set(num_workers=2)

In [ ]:
import pandas as pd

In [ ]:
af1 = malariagen_data.Af1(pre=True,
                          gcs_cache='/home/namulil/lstm_projects/funestus_llineup/gcs_cache',
                          results_cache='home/namulil/lstm_projects/funestus_llineup/results_cache'
                          )

In [ ]:
# run this code to read in the additional sample metadata

additional_metadata = pd.read_csv('llineup_funestus_extra_metadata.csv')
additional_metadata['partner_sample_id'] = additional_metadata['partner_sample_id'].astype(str)

In [ ]:
# run this code to add the additional metadata to the sample metadata available in the package
af1.add_extra_metadata(additional_metadata, on='partner_sample_id')

In [ ]:
# run this code to retrieve a dataframe containing the combined sample metadata
llineup_sample_set = '1288-VO-UG-DONNELLY-VMF00219'
df_samples = af1.sample_metadata(llineup_sample_set)

In [ ]:
from collections import Counter
import numpy as np
import plotly.express as px
import bokeh.io as bkio
bkio.output_notebook()
import os
import pandas as pd
import ipyleaflet
import allel
import matplotlib.pyplot as plt

In [ ]:
pip install plotly==5.18.0

In [ ]:

def tajimas_d_over_genome(self, 
                          cohort_label="cohort_armpp_round", 
                          region="2RL:60,000,000-80,000,000", 
                          df_samples=df_samples, 
                          sample_sets=llineup_sample_set, 
                          window_size=20_000):

    sample_queries = {v: f"{cohort_label} == '{v}'" for v in df_samples[cohort_label]}
    tajimas_ds = []
    cohorts = []
    
    for cohort, query in sample_queries.items():
        ac = self.snp_allele_counts(region=region, sample_sets=sample_sets, sample_query=query)
        pos = self.snp_sites(region=region, field='POS')
        
        # Calculate Tajima's D
        tajimas_d = allel.windowed_tajima_d(ac=ac, pos=pos, size=window_size)
        tajimas_ds.append(tajimas_d[0])
        
        cohorts.append(cohort)

    return cohorts, tajimas_ds



TAJIMA'S D

In [ ]:
cohorts, tajimas_ds = tajimas_d_over_genome(
    self=af1, 
    cohort_label="cohort_armpp_round", 
    region="2RL:60,000,000-80,000,000", 
    sample_sets='1288-VO-UG-DONNELLY-VMF00219', 
    window_size=20_000
)


In [ ]:
dif_samples = pd.DataFrame (tajimas_ds, index=cohorts)
dif_samples =dif_samples.T

In [ ]:
dif_samples.head(3)

In [ ]:
dif_samples=  dif_samples.melt(var_name='cohort_armpp_round', value_name='Tajimas')

In [ ]:
dif_samples

In [ ]:
pi_df = dif_samples.merge(df_samples)

In [ ]:
pi_df

In [ ]:
rnd_dict = {'round_1':'RND == 1', 'round_2':'RND == 2', 'round_3':'RND == 3', 'round_4':'RND == 4', 'round_5':'RND == 5'}

{f'round_{i}':f'RND == {i}' for i in np.arange(1,6)}

In [ ]:
pip install kaleido

In [ ]:
import kaleido

In [ ]:
# you will need kaleido installed
fig = ppx.box(pi_df, x='cohort_armpp_round', y='Tajimas', color='RND', template="plotly_white")
fig.write_image("TajimasD.svg", scale=2) 



TAJIMA'S D